In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
import jax.numpy as jnp
from jaxley-mini.modules.comp import Compartment
from jaxley-mini.mechansims.channels.channel import Channel

ModuleNotFoundError: No module named 'modules'

In [ ]:
comp = Compartment()
comp.insert(Leak())
comp.insert(Na())
comp.insert(K())
comp.stimulate(jnp.concatenate([jnp.zeros(1000), jnp.ones(1000)*0.1, jnp.zeros(1000)]))

u0 = comp.states
for name, channel in comp.channels.items():
    u0[name] = channel.init(u0, comp.params)

u0["t"] = 0
p = comp.params
i_ext = comp.externals["i"]

def step(u, i_ext, dt = 0.025):
    u["i"] = i_ext
    u["v"] = u["v"] + comp.dV(u, p, u["t"]) * dt
    for name, channel in comp.channels.items():
        for k, du in channel.du({**u, **u[name]}, {**p, **p[name]}, u["t"]).items():
            u[name][k] = u[name][k] + du * dt
    u["t"] += dt
    u.pop("i")
    return u, u

_, u = jax.lax.scan(step, u0, i_ext)
